In [1]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

In [2]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-16 20:45:37.900467: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 20:45:38.688392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("./pr_ml_nd_cpe_and_access_vector.csv")

In [5]:
import random
all_data = [ [el[0],[el[1:]]] for el in data.values.tolist()]
random.shuffle(all_data)
X=[el[0] for el in all_data]
y = [el[1][0] for el in all_data]

In [6]:
X=np.array(tokenizer(X, padding="max_length",truncation=True,max_length=192)["input_ids"])


In [7]:
y=np.array(y,dtype=np.float32)

In [10]:
# with open('X_4.npy', 'wb') as f:
#     np.save(f, X)
# with open('y_4.npy', 'wb') as f:
#     np.save(f, y)

In [11]:
# with open('X_1.npy', 'rb') as f:
#     X = np.load(f)
# with open('y_1.npy', 'rb') as f:
#     y = np.load(f)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,shuffle=True)

In [13]:
y_train = np.array(y_train,np.float32)
y_test  = np.array(y_test,np.float32)

In [14]:
# import torch 
# X_train_cuda = torch.from_numpy(X_train).cuda()
# X_test_cuda = torch.from_numpy(X_test).cuda()
# y_test_cuda = torch.from_numpy(y_test).cuda()
# y_train_cuda = torch.from_numpy(y_train).cuda()

In [15]:
from transformers import RobertaModel, AutoConfig
configuration = AutoConfig.from_pretrained('roberta-base')
configuration.hidden_dropout_prob = 0.3
configuration.attention_probs_dropout_prob = 0.3
configuration.num_labels=4
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config=configuration)
# model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", config=configuration)
model=model.to("cuda")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should pr

In [16]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, tokens,labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        tokens = self.tokens[idx]
        label = self.labels[idx]
        try:
            return (tokens, label)
        except Exception as e:
            print(e)


In [17]:
import torch
BATCH_SIZE=42
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,shuffle=True,pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=True,pin_memory=True)

In [18]:
print(len(train_dataloader),len(test_dataloader))

4399 1467


In [28]:
from tqdm import tqdm
from timeit import default_timer as timer
from statistics import mean
from torch.cuda.amp import GradScaler
from torch import autocast
# import optuna
# import wandb
import numpy as np
from sklearn.metrics import accuracy_score
device="cuda"

criterion = torch.nn.BCEWithLogitsLoss()

def train(lr,trial=None,EPOCHS=5):
    # wandb.init(project="bert_test", entity="qwertyforce",reinit=True,name="123_test")

    # wandb.config.update({
    #     "learning_rate": lr,
    #     "epochs": EPOCHS,
    #     "batch_size": BATCH_SIZE,
    #     })
    wandb=None
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr) 
    scaler = GradScaler()

    loss_train=[]
    loss_test=[]
    acc_test=[]

    for epoch in tqdm(range(EPOCHS)):
        train_loss = train_one_epoch(optimizer,criterion,scaler)
        loss_train.append(train_loss)

        test_loss,test_acc = test(criterion)
        loss_test.append(test_loss)
        acc_test.append(test_acc)

        if wandb:
            wandb.log({"loss_train": loss_train[-1],"epoch":epoch})
            wandb.log({"loss_test": loss_test[-1],"epoch":epoch})
            wandb.log({"acc_test": acc_test[-1],"epoch":epoch})
        # if trial:
        #     trial.report(mean(loss_test), epoch)
        #     if trial.should_prune():
        #         raise optuna.exceptions.TrialPruned()
    return loss_train,loss_test,acc_test


def train_one_epoch(optimizer,criterion,scaler):
    model.train()
    temp_train_loss=[]
    start = timer()
    for batch_idx, (data, labels) in enumerate(train_dataloader):
        data, labels = data.to(device), labels.to(device)
        # print(data.shape)
        # labels=labels.reshape(-1)
        # labels = labels.squeeze()
        # print(labels.shape)
        optimizer.zero_grad()
        
        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model.forward(data)["logits"]
            # print(outputs.dtype)
            # print(labels.dtype)
            loss = criterion(outputs,labels)
        
        # outputs = model.forward(data)
        # loss = criterion(outputs,labels.unsqueeze(1))
        # loss.backward()
        # optimizer.step()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        temp_train_loss.append(loss.item())
        if batch_idx % 100==0:
            if wandb:
                wandb.log({"loss_train_batch_idx": mean(temp_train_loss[-100:]),"batch_idx":batch_idx})

        if batch_idx % 500==0:
            if wandb:
                wandb.log({"loss_test_batch_idx": test(criterion,partial=True),"batch_idx":batch_idx})
                model.train()
            
    end = timer()
    train_loss = mean(temp_train_loss)
    print(f"Train loss = {train_loss}; epoch_training_time: {end - start}")
    return train_loss

            

def test(criterion,partial=False):
    model.eval()
    temp_loss=[]
    predictions, true_labels = [], []
    for batch_idx, (data, labels) in tqdm(enumerate(test_dataloader)):
        with torch.no_grad():
            # labels = labels.squeeze()
           
            true_labels.extend(labels.cpu().numpy())
            data, labels = data.to(device), labels.to(device)
            
            # labels=labels.reshape(-1)
            outputs = model.forward(data)["logits"]
            print(outputs.device)
            print(labels.device)
            loss = criterion(outputs,labels)
            temp_loss.append(loss.item())
              #   print(outputs)
             #   break
            outputs = torch.sigmoid(outputs).cpu().numpy()
            outputs = np.round(outputs)
            # outputs=outputs.cpu().numpy()
            # outputs=np.argmax(outputs,axis=1)
            # outputs = np.round(outputs)
            # print(outputs)
            predictions.extend(outputs)
            if partial and batch_idx == 100:
                return mean(temp_loss)

    accuracy = accuracy_score(true_labels,predictions)
    test_loss = mean(temp_loss)
    print(f"Test loss = {test_loss}; Test acc = {accuracy}")
    return test_loss, accuracy

In [ ]:
train(0.00002,EPOCHS=5)

In [ ]:
# a=[]
# b=[]
# test(a,b,criterion)

In [ ]:
model.eval()
temp_loss=[]
predictions, true_labels = [], []
for batch_idx, (data, labels) in tqdm(enumerate(test_dataloader)):
    with torch.no_grad():

        true_labels.extend(labels.cpu().numpy())
        data, labels = data.to(device), labels.to(device)
        # labels=labels.reshape(-1)
        outputs = model.forward(data)["logits"]
        loss = criterion(outputs,labels)
        temp_loss.append(loss.item())
    #   print(outputs)
    #   break
        outputs = torch.sigmoid(outputs).cpu().numpy()
        outputs = np.round(outputs)
        # outputs=outputs.cpu().numpy()
        # outputs=np.argmax(outputs,axis=1)
        # print(outputs)
        predictions.extend(outputs)
        # if batch_idx == 10:
        #     break


print(mean(temp_loss))
print(accuracy_score(true_labels,predictions))
print(f"Test: {mean(temp_loss)}")

963it [06:58,  2.30it/s]

0.10316330983921468
0.893150595953363
Test: 0.10316330983921468


In [ ]:
# rr=0
# for i in range(len(predictions)):
#     if (true_labels[i]==predictions[i]).all():
#         rr+=1

In [ ]:
# rr/len(predictions)

In [ ]:
# rrr=0
# for i in range(len(predictions)):
#     if true_labels[i][9]==predictions[i][9]:
#         rrr+=1
# print(rrr/len(predictions))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels,predictions))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98     54281
           1       0.87      0.81      0.84      3337
           2       0.82      0.84      0.83     13708
           3       0.75      0.66      0.70       671

   micro avg       0.94      0.94      0.94     71997
   macro avg       0.85      0.82      0.84     71997
weighted avg       0.94      0.94      0.94     71997
 samples avg       0.95      0.96      0.95     71997



/home/qwertyforce/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
